## Puente Snap - MQTT usando python
#### David Ochoa
#### ochoadavid at gmail.com
#### http://github.com/ochoadavid/

Desde hace algunos años se han desarrollado sistemas gráficos para ayudar en la enseñanza de la programación. Se destaca en este sentido __Scratch__ y sus derivados.

![Scratch](imagenes/Screenshot-ScratchPongGame1.png)

__SNAP__ (https://snap.berkeley.edu/) uno de sus derivados mantiene el mismo estilo de programación en un sistema que se ejecuta en el navegador y se puede ejecutar desde el sitio o simplemente descargandolo y abriendolo localmente.

In [1]:
from IPython.display import IFrame
IFrame('https://snap.berkeley.edu/', width=780, height=550)

Existen interfaces desarrolladas para interactuar con algunos dispositivos:

Use devices with Snap!:

* Orbotix Sphero guide by Connor Hudson and Dan Garcia
* Lego NXT package by Connor Hudson
* Nintendo Wiimote package by Connor Hudson
* Finch and Hummingbird robots package by Tom Lauwers
* Parallax S2 robot package by Connor Hudson
* LEAP Motion by Connor Hudson
* Speech synthesis by Connor Hudson
* Arduino package by Alan Yorinks
* Arduino package by Bernat Romagosa/Citilab
* Fischertechnik ROBOTICS TXT Controller by Richard Kunze
* Snap! for Raspberry Pi by rasplay.org

Sin embargo resulta interesante tener una interfaz genérica para enviar mensajes a un __servidor MQTT__ (referido como __broker__) para controlar dispositivos que utilicen este protocolo. 

Con este fin se desarrollo un “puente” en Python 3 que recibe mensajes de Snap y usando la librería paho-mqtt transmite estos mensajes para el servidor (broker). De ahí los dispositivos conectados al broker reciben los mensajes y actúan en consecuencia. En la platica se describirá la estructura de este puente, sus características y detalles del desarrollo. 

## MQTT

¿Qué es MQTT?

__MQTT__ (Message Queuing Telemetry Transport) es un protocolo de conectividad maquina-a-maquina. Fue diseñado como un transporte de mensajes a través de _publicar_ y _subscribir_. Es útil para conexiones remotas donde pueda haber limitaciones de ancho de banda y/o equipos con poder de computo limitado, por lo que resulta un muy buen candidato para ser utilizado para _Internet de las cosas_ (IOT).

In [2]:
from IPython.display import IFrame
IFrame('http://mqtt.org/', width=780, height=550)

### Conceptos básicos de MQTT:
    
* Publicar/suscribir (Publish/Subscribe)
* Mensajes (Messages)
* Temas (Topics)
* Servidor (Broker)

#### MQTT - Publicar/Suscribir

El protocolo MQTT es un protocolo donde un __Cliente__ puede realizar dos acciones:
* Publicar: el cliente manda un mensaje.

* Suscribir: el cliente pide ser informado cuando un __Tema__ sea publicado.  

#### MQTT – Mensajes

Los mensajes son los paquetes de información que queremos intercambiar entre diversos dispositivos. Pueden ser comandos o datos.

#### MQTT – Temas (Topic)

Los temas nos permiten categorizar los mensajes de tal manera que cuando un cliente publica estos sólo sean transmitidos a aquellos clientes subscritos a dicho tema.

Los temas se representan con cadenas (strings) seguidas por una diagonal (__/__). Cada diagonal es un __nivel__ de tema. Por ejemplo, el tema para una lampara en el escritorio podría ser:
## /escritorio/lampara

__Nota__: ¡MAYUSCULAS/minúsculas importan! (Escritorio $\ne$ escritorio)

#### MQTT – Servidor (Broker)

El servidor es el responsable de recibir los mensajes, filtrarlos y enviarlos a los clientes subscritos.

![MQTT](imagenes\mqtt1.png)

__Características:__

* Todos los clientes son iguales
* Todos los clientes pueden publicar y/o suscribirse
* El Broker concentra todos los mensajes y los distribuye a los clientes suscritos

__Servidores en la web__:
Algunos sitios web que ofrecen este servicio:
* http://www.maqiatto.com/
* https://www.hivemq.com/
* https://www.cloudmqtt.com/
* https://test.mosquitto.org/

(Todos estos tienen algún servicio gratuito y algunos servicios pagados)

__Servidores locales__:
Disponibles para descargar e instalar:
* Mosquitto: __broker__ escrito en C. (https://mosquitto.org/)
* Mosca: basado en Node.js. (https://github.com/mcollina/mosca)
* emqttd: escrito en Erlang. (https://github.com/emqx/emqx)
* Servidor de prueba en Python. (https://github.com/eclipse/paho.mqtt.testing/tree/master/interoperability)
* Verne. https://vernemq.com/

In [3]:
from IPython.display import IFrame
IFrame('http://mosquitto.org/', width=780, height=550)

## Puente Snap - MQTT

La idea del puente es muy sencilla (en realidad el puente es muy sencillo): snap tiene soporte nativo para mandar hacer un __request__ via http. La información del tema y el mensaje se pasa como variable al request:
![Bloque SNAP](imagenes/bloqueSNAP.png)

El puente funciona de la siguiente manera: al iniciar se conecta como __cliente__ al Broker, después inicia un __servior http__ donde recibe estos mensajes y los publica al __Broker MQTT__. 

__Librerías utilizadas__:

Además de las librerías para hacer un servidor __http__ sencillo utilizamos la librería `client` de `paho.mqtt`:

```
import http.server
import socketserver
import paho.mqtt.client as mqtt
```

__\_\_main\_\___:

El programa principal se encarga de hacer las conexiones y declarar quien se encargará de recibir los mensajes.

```
if __name__ == "__main__":
    print("Snap-MQTT")

    client = mqtt.Client()
    client.connect("192.168.4.1")

    client.on_connect = on_connect
    client.loop_start()

    handler = RequestHandler

    httpd = socketserver.TCPServer(("", 1330), handler)

    httpd.serve_forever()
```

__Aviso de conexión__:

El método `on_connect` se utiliza solamente para dar un aviso al usuario:

```
def on_connect(client, userdata, flags, rc):
    print('Conectado a broker')
```

__Receptor de mensajes__:

El objeto `handler` de la clase `RequestHandler` se encarga de recibir el mensaje y enviarlo al Broker:


```
class RequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        path = self.path[1:]
        topic, msg = path.split("?")
        print(topic, " - ", msg)
        client.publish(topic, msg, qos=1)
        self.send_response(204)
        self.end_headers()
```

## Cliente suscrito a los mensajes

El cliente que recibe el mensaje, al igual que el servidor __Mosquitto__, se encuentra en el RaspberryPi.

__Librería utilizada__:

Unicamente es importada la librería `client` de `paho.mqtt` (en realidad también es utilizada `DigitalOutputDevice` de la librería `gpiozero`, pero como es propia del RaspberryPi, será importada en otro lugar)

```
import paho.mqtt.client as mqtt
```

__\_\_main\_\___:


Después de declarar un estado inicial de los motores se conecta al Broker y se sobreescriben dos funciones de la clase. Finalmente se inicia un ciclo `while True:` que en cada iteración llama `client.loop(0.02)` y una función llamada monitor. 

```
if __name__ == "__main__":
    estado = {'pos0':0, 'pos1':0, 'pobj0':0, 'pobj1':0}

    client = mqtt.Client()
    client.connect("localhost")

    client.on_connect = on_connect
    client.on_message = on_message

    while True:
        client.loop(0.02)
        mueve(estado)
```

__client.loop(0.02)__

Permite al cliente tomar el control de la computadora por `0.02` segundos (enviando y recibiendo los mensajes necesarios).

__Función `on_connect()`__:

En este caso la función `on_connect` se suscribe a los __temas__ que nos interesan.


```
def on_connect(client, userdata, flags, rc):
    print("Conectado a broker")
    client.subscribe("rpi/motor0")
    client.subscribe("rpi/motor1")
```

__Función `on_message()`__:


Esta función es llamada cada que un mensaje llega al cliente (de aquellos a los que esta suscrito). La función actualiza las posiciones objetivo del estado cuando esto sucede.

```
def on_message(client, userdata, msg):
    global estado
    if msg.topic == "rpi/motor0":
        estado['pobj0'] = int(msg.payload.decode())
    elif msg.topic == "rpi/motor1":
        estado['pobj1'] = int(msg.payload.decode())

```

__Función `mueve()`__:

Esta función es la que ejecuta el movimento en cada ciclo.


```
pmap = [[True, True, False, False],
        [False, True, True, False],
        [False, False, True, True],
        [True, False, False, True]]

def mueve(es):
    if es['pos0'] != es['pobj0']:
        pin = es['pos0'] % 4
        if es['pos0'] < es['pobj0']:
            es['pos0'] += 1
        elif es['pos0'] > es['pobj0']:
            es['pos0'] -= 1
        pout = es['pos0'] % 4
        print('M0={}, obj={}'.format(es['pos0'], es['pobj0']))
        for m, pi, po in zip(motor0, pmap[pin], pmap[pout]):
            if po and not pi:
                m.on()
            elif not po and pi:
                m.off()

```

Para controlar un motor de pasos unipolar utilizamos cuatro salidas del gpio en forma de una lista, `motor0`, que contiene estas cuatro salidas:


```
from gpiozero import DigitalOutputDevice as Dod
m0a = Dod(2, active_high=False)
m0b = Dod(3, active_high=False)
m0c = Dod(4, active_high=False)
m0d = Dod(17, active_high=False)
motor0 = [m0a, m0b, m0c, m0d]
```

### Demostración

### Ideas finales

* ¡MQTT es realmente sencillo de usar!

* IOT -> "Internet de los programas!"

* Seguridad

### ¿Preguntas?

### ¡Gracias!
#### David Ochoa
#### ochoadavid at gmail.com
#### http://github.com/ochoadavid/